# Models of Inverse Beta Decay

The interactions module in ASTERIA contains two implementations of inverse beta decay:

1. A simple parametric model.
2. A tabular model based on a full calculation of the interaction.

Both calculations can be found in [Strumia and Vissani, Phys. Lett. B 564:42, 2003](https://www.sciencedirect.com/science/article/pii/S0370269303006166).

In [ ]:
from asteria import set_rcparams
from asteria.interactions import InvBetaPar, InvBetaTab
from snewpy.neutrino import Flavor

import astropy.units as u

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

## Setup styles for Plotting

In [ ]:
set_rcparams(verbose=True)

## Compare Cross Sections and Mean Energies

For the energies of interest to IceCube, the parametric and tabular models differ by a few percent in cross section and mean energy.

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(12, 6), sharex=True,
                         gridspec_kw={'height_ratios':[3,1], 'hspace':0, 'wspace': 0.25})

xs_old = None
lep_old = None
enu = np.linspace(0., 200., 101) * u.MeV

for ibd, style, lab in zip([InvBetaPar(), InvBetaTab()],
                           ['-', '.'],
                           ['Parametric (Eq. 25)', 'Table 1']):
    xs = ibd.cross_section(Flavor.NU_E_BAR, enu)
    lep = ibd.mean_lepton_energy(Flavor.NU_E_BAR, enu)
    axes[0][0].plot(enu, xs, style, label=lab)
    axes[0][1].plot(enu, lep, style, label=lab)

    # Plot residuals
    if xs_old is not None and lep_old is not None:
        cut = xs_old != 0.
        res = (xs[cut] - xs_old[cut])/xs_old[cut]
        axes[1][0].plot(enu[cut], 1e2*res, style)
        cut = lep_old != 0.
        res = (lep[cut] - lep_old[cut])/lep_old[cut]
        axes[1][1].plot(enu[cut], 1e2*res, style)
    xs_old = xs
    lep_old = lep

axes[0,0].set_ylabel( r'$\sigma(\bar{\nu}_e p\rightarrow e^{+}n)$ [cm$^2$]', horizontalalignment='right', y=1)
axes[0,0].set(yscale='log')
axes[1,0].set_xlabel( r'$E_\nu$ [MeV]', horizontalalignment='right', x=1)
axes[1,0].set_ylabel( r'$\Delta\sigma/\sigma$', horizontalalignment='right', y=1)
axes[1,0].set(ylim=[-1.2,1.2])
axes[0,1].set_ylabel( r'$\langle E_e\rangle$ [MeV]', horizontalalignment='right', y=1)
axes[1,1].set_xlabel( r'$E_\nu$ [MeV]', horizontalalignment='right', x=1)
axes[1,1].set_ylabel( r'$\Delta\langle E\rangle/\langle E\rangle$', horizontalalignment='right', y=1)

axes[1,1].set(ylim=[-5,15])

leg = axes[0,0].legend()
fig.suptitle(r'Inv. $\beta$ Models from Strumia and Vissani, PLB 564, 2003')
fig.subplots_adjust(left=0.025, right=0.975)